In [1]:
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras 
import warnings

warnings.filterwarnings('ignore')
print(tf.__version__)
for model in sklearn, pd, keras, np ,mlp:
    print(model.__name__, model.__version__)

2.1.0
sklearn 0.20.2
pandas 0.24.2
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
numpy 1.17.4
matplotlib 2.1.2


In [2]:
# https://storage.googleapis.com/tf-datasets/taitanic/train.csv
# https://storage.googleapis.com/tf-datasets/taitanic/eval.csv
train_file = "./data/titanic/train.csv"
eval_file = "./data/titanic/eval.csv"
train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)
train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_df = train_df.fillna(method='ffill')
train_df = train_df.dropna(axis=0, how='any')
eval_df = eval_df.fillna(method='ffill').dropna(axis=0, how="any")
print(len(eval_df))
print(train_df.shape)

232
(655, 12)


In [4]:
y_train  = train_df.pop('Survived')
y_eval  = eval_df.pop('Survived')
print(len(y_train))
print(len(y_eval))

655
232


In [5]:
# PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
categorical_columns = ['Sex', 'SibSp', 'Parch', 'Pclass', 'Embarked']
numeric_columns = ['Age', 'Fare']
feature_columns = []

for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    print(categorical_column, vocab)
    feature_columns.append(
        # 进行one-hot
        tf.feature_column.indicator_column(
            # 将所有可能出现的vocabulary进行分列
            tf.feature_column.categorical_column_with_vocabulary_list(categorical_column, vocab)
        )
    )
    
for numeric in numeric_columns:
    print(numeric)
    feature_columns.append(
        tf.feature_column.numeric_column(numeric, dtype=tf.float32)
    )
    
# 添加交叉特征
feature_columns.append(
    tf.feature_column.indicator_column(
        tf.feature_column.crossed_column(['Age', "Sex"], hash_bucket_size=100)
    )
)
print(feature_columns)

Sex ['female' 'male']
SibSp [1 0 3 4 2 5 8]
Parch [0 1 2 5 3 4]
Pclass [1 3 2]
Embarked ['C' 'S' 'Q']
Age
Fare
[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Sex', vocabulary_list=('female', 'male'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='SibSp', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Pclass', vocabulary_list=(1, 3, 2), dtype=tf.int64, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Embarked', vocabulary_list=('C', 'S', 'Q'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), NumericColumn(key='Age', shape=(1,), d

In [6]:
def make_dataset(data_df, label_df, epochs=10, shuffle=True, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [7]:
# base line
output_dir = "baseline_model"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
baseline_model = tf.compat.v1.estimator.BaselineClassifier(model_dir=output_dir, n_classes=2)
baseline_model.train(input_fn=lambda :make_dataset(train_df, y_train, epochs=100))


W0223 23:45:12.446131 4757194176 deprecation.py:506] From /anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0223 23:45:12.447631 4757194176 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow_core/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0223 23:45:12.794347 4757194176 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow_core/python/training/saver.py:1069: get_checkpoint_mtimes (from tensorflo

In [8]:
baseline_model.evaluate(input_fn=lambda :make_dataset(eval_df, y_eval, epochs=1, shuffle=False, batch_size=20))

W0223 23:45:17.274022 4757194176 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/metrics_impl.py:2029: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0223 23:45:17.349261 4757194176 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/canned/head.py:619: auc (from tensorflow.python.ops.metrics_impl) is deprecated and will be removed in a future version.
Instructions for updating:
The value of AUC returned by this may race with the update so this is deprected. Please use tf.keras.metrics.AUC instead.
W0223 23:45:17.584880 4757194176 metrics_impl.py:806] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0223 23:45:17.598378 4757194176 metrics_impl.py:806] Trapezoidal rule is known to produce incorrect PR-AUCs; plea

{'accuracy': 0.63793105,
 'accuracy_baseline': 0.63793105,
 'auc': 0.5,
 'auc_precision_recall': 0.68103445,
 'average_loss': 0.65676916,
 'label/mean': 0.36206895,
 'loss': 12.697536,
 'precision': 0.0,
 'prediction/mean': 0.3940946,
 'recall': 0.0,
 'global_step': 8188}

In [9]:
baseline_model.evaluate(input_fn=lambda :make_dataset(eval_df, y_eval, epochs=1, shuffle=False, batch_size=20))

W0223 23:45:20.042619 4757194176 metrics_impl.py:806] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0223 23:45:20.056655 4757194176 metrics_impl.py:806] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.


{'accuracy': 0.63793105,
 'accuracy_baseline': 0.63793105,
 'auc': 0.5,
 'auc_precision_recall': 0.68103445,
 'average_loss': 0.65676916,
 'label/mean': 0.36206895,
 'loss': 12.697536,
 'precision': 0.0,
 'prediction/mean': 0.3940946,
 'recall': 0.0,
 'global_step': 8188}

In [11]:
# Linear Model
linear_model = "linear_model_new"
linear_model = tf.compat.v1.estimator.LinearClassifier(model_dir=linear_model, n_classes=2, feature_columns=feature_columns)
linear_model.train(input_fn=lambda :make_dataset(train_df, y_train, epochs=100))

W0223 23:48:17.513892 4757194176 base_layer.py:1790] Layer linear_model is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [12]:
linear_model.evaluate(input_fn=lambda:make_dataset(eval_df, y_eval, epochs=1, batch_size=20, shuffle=False))

W0223 23:48:27.302957 4757194176 base_layer.py:1790] Layer linear_model is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W0223 23:48:27.923924 4757194176 metrics_impl.py:806] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0223 23:48:27.938685 4757194176 metrics_impl.py:806] Trapezoidal rule is known to produce

{'accuracy': 0.8448276,
 'accuracy_baseline': 0.63793105,
 'auc': 0.88408947,
 'auc_precision_recall': 0.85553175,
 'average_loss': 0.3950442,
 'label/mean': 0.36206895,
 'loss': 7.6375213,
 'precision': 0.77272725,
 'prediction/mean': 0.40724537,
 'recall': 0.8095238,
 'global_step': 2047}

In [13]:
# dnn model
dnn_dir = "dnn_model_new"
if not os.path.exists(dnn_dir):
    os.mkdir(dnn_dir)
dnn_model = tf.estimator.DNNClassifier(model_dir=dnn_dir,
                                       n_classes=2,
                                       feature_columns=feature_columns,
                                       hidden_units=[128, 128],
                                       activation_fn=tf.nn.relu,
                                       optimizer='Adam')
dnn_model.train(input_fn=lambda : make_dataset(train_df, y_train, epochs=100))

W0223 23:48:33.968867 4757194176 base_layer.py:1790] Layer dnn is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [14]:
dnn_model.evaluate(input_fn=lambda:make_dataset(eval_df, y_eval, epochs=1, batch_size=20, shuffle=False))

W0223 23:48:37.911555 4757194176 base_layer.py:1790] Layer dnn is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



{'accuracy': 0.76724136,
 'accuracy_baseline': 0.63793105,
 'auc': 0.85714287,
 'auc_precision_recall': 0.7755078,
 'average_loss': 0.7129677,
 'label/mean': 0.36206895,
 'loss': 0.71630055,
 'precision': 0.65,
 'prediction/mean': 0.46332684,
 'recall': 0.77380955,
 'global_step': 2047}